# Outline of thought process
- starting out with the creature class as the base class. The creature class will be used to create monsters/creatures that<br>
have not become npcs

- the pc_class class will be used to give player characters access to class mechanics
- the pc class will inherit from the creature class and the pc_class class to create a pc
- the npc class will inherit from the creature class and set the is_npc flag of an object to True so that a monster can be <br>
changed to npc status and stored to come back as a recurring character
- the item class will be used to create all items(including interactable objects our pcs might run into or use to change the <br> course of a battle. Think setting a fire to obscure the battle field or wedging a door shut


In [ ]:
import pc_classes
import creatures
import races
import copy

from dice import Dice, DiceBag, dice_avg
from skills import skills
from weapons import weapons as wpn
from armor import armor as ar

In [ ]:
### start of a creature class

class Creature():
    def __init__(self, entity_type, creature):
        creature_attrs = creatures.creature_keys
        self.entity_type = entity_type
        if entity_type == 'pc':
            self.disposition = 'ally'
            self.hp_max = 0
        for key in creature.keys():
            setattr(self, key, creature[key])
        for attr in creature_attrs:
            if hasattr(self, attr)==True:
                pass
            else:
                setattr(self, attr, False)
                
    def __dual_weilder_check(self):
        dual_weilder = False
        for prof in self.proficiencies:
            if 'dual_weilder' in prof:
                dual_weilder = True
        return dual_weilder
    
    def __armor_disadvantage(self):
        if self.disadvantage == False:
            self.disadvantage = []
            
        self.disadvantage.append('dex')
        self.disadvantage.append('str')
        self.disadvantage.append('attack')
        self.spell_casting = False
        for skill in self.skills:
            if self.skills[skill]['ability']== 'str' or self.skills[skill]    ['ability']== 'dex':
                self.disadvantage.append(skill)
                
    def equip_armor(self, armor):
        ac_base = armor['armor_class']['base']
        ac_dex_bon = armor['armor_class']['dex_mod']
        ac_dex_max = armor['armor_class']['dex_mod_max']
        ac_str_req = armor['str_req']
        ac_stealth = armor['stealth_penalty']
        
        if self.armor == False:
            self.armor = armor['name']
            if ac_str_req != False:
                self.ac = ac_base
                if self.abilities['str'] < ac_str_req:
                    self.movement -= 10
            if ac_dex_bon == True:
                if ac_dex_max != False:
                    self.ac = ac_base + ac_dex_max
                else:
                    self.ac = ac_base + creatures.ability_mod_tbl[self.abilities['dex']]
            else:
                self.ac = ac_base
            
            for prof in self.proficiencies:
                if armor['type'] in prof:
                    pass
            else:
                self.__armor_disadvantage()

        else:
            print(self.armor + ' is already equiped.')
    
    def equip_shield(self, shield):
        if self.off_hand == False:
            self.ac += 2
            self.off_hand == shield['name']
            for prof in self.proficiencies:
                if shield['type'] in prof:
                    pass
            else:
                self.__armor_disadvantage()
        else:
            print(self.off_hand + ' is already equiped.')
            
    
    def equip_weapon(self, weapon, main_hand = True, off_hand = False, 
                     versatile=False,):
        dual_weilder = self.__dual_weilder_check()

        #for prof in self.proficiencies:
        #    if 'dual_weilder' in prof:
        #        dual_weilder = True
        if off_hand == False and versatile == False:
            if self.main_hand != False:
                print(self.main_hand + ' is already equiped')
            else:
                if 'two_handed' in weapon['properties']:
                    self.main_hand = weapon['name']
                    self.off_hand = weapon['name']
                else:   
                    self.main_hand = weapon['name']
        if versatile == True:
            if self.off_hand != False:
                print(self.off_hand + 
                      ' is equipped in off_hand. Please unequip to use this weapons versatile properties')
            elif self.main_hand != False or self.main_hand != weapon['name']:
                print(self.main_hand +
                     ' is equipped. Please unequip this weapon to equip a new weapon')
            else:
                self.main_hand = weapon['name']
                self.off_hand = weapon['name']
                
        if off_hand == True:
            if self.off_hand != False:
                print(self.off_hand + ' is already equiped')
            elif self.main_hand != False:
                if'light' not in wpn[self.main_hand]['properties'] and dual_weilder == False:
                    print('You do not meet the requirements for dual weilding')
            elif 'light' not in weapon['proficiencies'] and dual_weilder == False:
                 print('You do not meet the requirements for dual weilding')
            else:
                self.off_hand = weapon['name']
                    
                
                
            
            
    #def un-equip_armor(self)
    #def attack(self, weapon):
    #def add_inventory(self, itm):
    #def cast(self, spell):
        #if spell.r_attack == True:
            #attack(spell)
        #elif spell.r_save == True:
        #else:
            #do a thing
    #def attack(self, weapon):
        #if weapon.a_type == melee:
            #do some things
        #elif weapon.a_type == ranged:
            #do some things
    #def dash(self, movement):
    #def disengage(self):
    #def dodge(self.advantage_toggle):
    #def help(self,target):
    #def hide(self, stealth_check):
    #def ready(self, trigger, action):
    #def search(self, perception_check):
    #def use_an_object(self):
    #def imporovise(self, skill_check):

In [ ]:
class Pc(Creature):
    def __init__(self):
        pass
    def create_pc(self, creature):
        setattr(creature, 'pc', True)
        setattr(creature, 'pc_lvl', 1)
        setattr(creature, 'proficiency_bonus', pc_classes.prof_bonus[creature.pc_lvl])
        creature.skills = copy.deepcopy(skills)
        if creature.armor == False and creature.ac == False:
            dex_mod = creatures.ability_mod_tbl[creature.abilities['dex']]
            wis_mod = creatures.ability_mod_tbl[creature.abilities['wis']]
            con_mod = creatures.ability_mod_tbl[creature.abilities['con']]
            barb_ua = 10 + con_mod + dex_mod
            monk_ua = 10 + dex_mod + wis_mod
            if 'barbarian' in creature.pc_classes.keys() & 'monk' in creature.pc_classes.keys():
                unarmored_defense = []
                unarmored_defense.append(barb_ua)
                unarmored_defense.append(monk_ua)
                creature.ac = max(unarmored_defense)
            elif 'barbarian' in creature.pc_classes.keys():
                creature.ac = barb_ua
            elif 'monk' in creature.pc_classes.keys():
                creature.ac = monk_ua
            else:
                creature.ac = 10 + creatures.ability_mod_tbl[creature.abilities['dex']]
        

In [ ]:
# keeping pc_class and pc as different classes to account for multi-classing

class PcClass:
    def __init__(self):
        pass
    
    def __hill_dwarf_hp_bonus(self, creature):
        creature.hp_max += 1
        
    def add_first_class(self, creature, pc_class, gen_method):
        p_class = pc_class['pc_class']
        setattr(creature, 'pc_classes', {p_class:1})
        setattr(creature, 'abilities', gen_method)
        
        ### adds first class attrs to creature
        for key in pc_class.keys():
            if key=='pc_class':
                pass
            else:
                setattr(creature, key, pc_class[key])
                
        ### adds racial bonuses to ability scores 
        for key in creature.ability_score_increase.keys():
            creature.abilities[key] += creature.ability_score_increase[key]
        
        #Turns creature with class into pc
        Pc.create_pc(self, creature)
        
        #move class profs into creature profs
        for prof in creature.arm_weap_prof:
            creature.proficiencies.append(prof)
            
        
        ### sets hp_max
        con_bonus = creatures.ability_mod_tbl[creature.abilities['con']]
        hp_max = int(pc_class['hit_die'][1:]) + con_bonus
        creature.hp_max = hp_max
        if creature.subrace == 'hill':
            self.__hill_dwarf_hp_bonus(creature)
            
    def add_class_lvl(self, creature, p_class_name):
        ### update hp_max
        class_hp_add = dice_avg[creature.hit_die]
        con_bonus = creatures.ability_mod_tbl[creature.abilities['con']]
        creature.hp_max += class_hp_add + con_bonus
        if creature.subrace == 'hill':
            self.hill_dwarf_hp_bonus(creature)
        
        creature.pc_classes[p_class_name] += 1
        creature.pc_lvl += 1
        creature.proficiency_bonus = pc_classes.prof_bonus[creature.pc_lvl]


In [ ]:
class Npc(Creature):
    def __init__(self):
        pass
    def is_npc(self):
        setattr(self, 'npc', True)
        setattr(creature, 'max_hp', 0)

In [ ]:
class item:
    def __init__(self):
        pass
    

In [ ]:
class Combat:
    def __init__(self, combatants):
        self.round = 1
        self.combatants = combatants
        #self.c_area = c_area
    
#    def initiative(self.combatants):
#        for c in self.combatants:
#            setattr(c, 'initiative', d20() + c.initiative_bonus)
    
        

In [ ]:
tiny = Creature('pc', races.dwarf_hill)
pc_class = PcClass()

In [ ]:
pc_class.add_first_class(tiny, pc_classes.rogue, pc_classes.rogue['ability_by_method']['pb']['main'])

In [ ]:
tiny.disadvantage

In [ ]:
tiny.equip_armor(ar['splint'])

In [ ]:
tiny.ac

In [ ]:
tiny.equip_shield(ar['shield'])

In [ ]:
tiny.ac

In [ ]:
tiny.disadvantage

In [ ]:
tiny.skills

In [ ]:
### consume spell slot command

In [ ]:
tiny.armor

In [ ]:
tiny.equip_weapon(wpn['longsword'])

In [ ]:
tiny.main_hand

In [ ]:
tiny.equip_weapon(wpn['shortsword'], off_hand = True)